This notebook shows how to train a neural network model with pre-trained BART in Tensorflow/Keras. It is based on @xhlulu's [Disaster NLP: Keras BERT using TFHub](https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub
) notebook.

This competition is a code competition without access to internet. So we add the `transformers` tokenizer and pre-trained BART base model through Kaggle Datasets instead. You can find the `transformers` tokenizer and pre-trained models for TF/Keras as follows:
* [TF BERT Base Uncased Pretrained Model/Tokenizer](https://www.kaggle.com/jeongyoonlee/tfbert-base-uncased)
* [TF BERT Large Uncased Pretrained Model/Tokenizer](https://www.kaggle.com/jeongyoonlee/tfbert-large-uncased)
* [TF RoBERTa Base Model/Tokenizer](https://www.kaggle.com/jeongyoonlee/tfroberta-base)
* [TF RoBERTa Lare Model/Tokenizer](https://www.kaggle.com/jeongyoonlee/tfroberta-large)
* [TF BART Base Pretrained Model/Tokenizer](https://www.kaggle.com/jeongyoonlee/tfbart-base)
* [TF BART Large Pretrained Model/Tokenizer](https://www.kaggle.com/jeongyoonlee/tfbart-large)

Please check out my other notebooks with the same code structure as follows:
* [PyTorch Lightning RoBERTa (Training/Inference)](https://www.kaggle.com/jeongyoonlee/pytorch-lightning-roberta-training-inference): PyTorch Lightning RoBERTa using `transformers`
* [TF/Keras BERT Baseline (Training/Inference)](https://www.kaggle.com/jeongyoonlee/tf-keras-bert-baseline-training-inference): TF/Keras BERT using TFHub (-v8) and `transformers` (v10+)

Hope it helps.

# Changelogs

| Version  | CV Score | Public Score | Changes | Comment |
|----------|----------|--------------|---------|---------|
|  v7 | TBD     | TBD | initial baseline | |

# Load Libraries and Data

In [ ]:
from copy import copy
import gc
import joblib
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import sys
from warnings import simplefilter

import tensorflow as tf
from tensorflow.keras import Model, Input, backend as K
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from transformers import TFBartModel, BartConfig, BartTokenizerFast

simplefilter('ignore')
plt.style.use('fivethirtyeight')

In [ ]:
# limit the GPU memory growth
gpu = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu))
if len(gpu) > 0:
    tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
model_name = 'tfbart_v7'

data_dir = Path('../input/commonlitreadabilityprize')
train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

build_dir = Path('./build/')
output_dir = build_dir / model_name
trn_encoded_file = output_dir / 'trn.enc.joblib'
val_predict_file = output_dir / f'{model_name}.val.txt'
submission_file = 'submission.csv'

pretrained_dir = Path('../input/tfbart-base')

id_col = 'id'
target_col = 'target'
text_col = 'excerpt'

max_len = 205
n_fold = 5
n_est = 9
n_stop = 2
batch_size = 8
seed = 42

In [ ]:
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
trn = pd.read_csv(train_file, index_col=id_col)
tst = pd.read_csv(test_file, index_col=id_col)
y = trn[target_col].values
print(trn.shape, y.shape, tst.shape)
trn.head()

# Tokenization Using `transformers`

In [ ]:
# For local experiments, download the pretrained tokenizer and model if not available.
pretrained_dir.mkdir(exist_ok=True)

def load_tokenizer():
    if not os.path.exists(pretrained_dir / 'vocab.json'):
        print('downloading the pretrained tokenizer')
        tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base")
        tokenizer.save_pretrained(pretrained_dir)
    else:
        print('loading the saved pretrained tokenizer')
        tokenizer = BartTokenizerFast.from_pretrained(str(pretrained_dir))
        
    config = BartConfig.from_pretrained(str(pretrained_dir))
    config.output_hidden_states = True
    return tokenizer, config

def load_bart(config):
    if not os.path.exists(pretrained_dir / 'tf_model.h5'):
        print('downloading the pretrained model')
        model = TFBartModel.from_pretrained("facebook/bart-base", config=config)
        model.save_pretrained(pretrained_dir)
    else:
        print('loading the saved pretrained model')
        model = TFBartModel.from_pretrained(pretrained_dir, config=config)
    return model

In [ ]:
def bart_encode(texts, tokenizer, max_len=max_len):
    input_ids = []
    attention_mask = []
    
    for text in texts:
        token = tokenizer(text, max_length=max_len, truncation=True, padding='max_length',
                         add_special_tokens=True)
        input_ids.append(token['input_ids'])
        attention_mask.append(token['attention_mask'])
    
    return np.array(input_ids), np.array(attention_mask)

In [ ]:
tokenizer, config = load_tokenizer()

X = bart_encode(trn[text_col].values, tokenizer, max_len=max_len)
X_tst = bart_encode(tst[text_col].values, tokenizer, max_len=max_len)
y = trn[target_col].values
print(X[0].shape, X_tst[0].shape, y.shape)

## Save Encoded Training Data

In [ ]:
joblib.dump(X, trn_encoded_file)

# Model Training with Cross-Validation

Simple model with only an output dense layer added to the pre-trained BERT model.

In [ ]:
def build_model(bart_model, max_len=max_len):    
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    sequence_output = bart_model(input_ids, attention_mask=attention_mask)[0]
    clf_output = sequence_output[:, 0, :]
    clf_output = Dropout(.1)(clf_output)
    out = Dense(1, activation='linear')(clf_output)
    
    model = Model(inputs=[input_ids, attention_mask], outputs=out)
    model.compile(Adam(lr=1e-5), loss='mean_squared_error', metrics=[RootMeanSquaredError()])
    
    return model

Training the model with early stopping and a learning-rate scheduler

In [ ]:
def scheduler(epoch, lr, warmup=5, decay_start=10):
    if epoch <= warmup:
        return lr / (warmup - epoch + 1)
    elif warmup < epoch <= decay_start:
        return lr
    else:
        return lr * tf.math.exp(-.1)

ls = LearningRateScheduler(scheduler)
es = EarlyStopping(patience=n_stop, restore_best_weights=True)

cv = KFold(n_splits=n_fold, shuffle=True, random_state=seed)

p = np.zeros_like(y, dtype=float)
p_tst = np.zeros((X_tst[0].shape[0], ), dtype=float)
for i, (i_trn, i_val) in enumerate(cv.split(X[0]), 1):
    print(f'training CV #{i}:')
    tf.random.set_seed(seed + i)
    
    bart_model = load_bart(config)
    clf = build_model(bart_model, max_len=max_len)
    if i == 1:
        print(clf.summary())

    history = clf.fit([x[i_trn] for x in X], y[i_trn],
                      validation_data=([x[i_val] for x in X], y[i_val]),
                      epochs=n_est,
                      batch_size=batch_size,
                      callbacks=[ls])
    clf.save_weights(f'{model_name}_cv{i}.h5')
    
    p[i_val] = clf.predict([x[i_val] for x in X]).flatten()
    p_tst += clf.predict(X_tst).flatten() / n_fold
    
    K.clear_session()
    del bart_model, clf
    gc.collect()

## Print CV RMSE and Save CV Predictions

In [ ]:
print(f'CV RMSE: {mean_squared_error(y, p, squared=False):.6f}')
np.savetxt(val_predict_file, p, fmt='%.6f')

# Submission

In [ ]:
sub = pd.read_csv(sample_file, index_col=id_col)
sub[target_col] = p_tst
sub.to_csv(submission_file)
sub.head()

If you find it helpful, please upvote the notebook. Also check out my other notebooks below:

* [TF/Keras BERT Baseline (Training/Inference)](https://www.kaggle.com/jeongyoonlee/tf-keras-bert-baseline-training-inference): shares the TF/Keras BERT baseline with 5-fold CV
* [PyTorch Lightning RoBERTa (Training/Inference)](https://www.kaggle.com/jeongyoonlee/pytorch-lightning-roberta-training-inference/notebook): shares the Pytorch Lightning baseline with 5-fold CV    
* [All Zero Submission](https://www.kaggle.com/jeongyoonlee/all-zero-submission): shows the public LB score for all zero submission
* [DAE with 2 Lines of Code with Kaggler](https://www.kaggle.com/jeongyoonlee/dae-with-2-lines-of-code-with-kaggler): shows how to generate Denoising AutoEncoder features using `Kaggler`

Happy Kagglging~!